### In this script we build line detector model using  Multilayer perceptron classifier in sklearn
- BY: Abdelraouf Hawash 
- DATE: 30 / 3 / 2022

### *import libraries*

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import pickle

### important attributes and methods

In [3]:
labels = ['QR','empty','horizontal','lef3','left2','left1','center','right1','right2','right3']

def preprocessing (img, dest_size = (20,20), dest_rang: int = 16):
    '''
    this function resize the image then makes pixels in a certain range
    it takes about 0.00035 s
    the input image should be in gray scale
    '''
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    out = (cv2.resize(img, dest_size) * (dest_rang/255)).astype('uint8')
    return out.reshape( out.shape[0] * out.shape[1] )

def show_processed_img (input, dest_size = (20,20), input_range: int = 16):
    out = (input.reshape(dest_size) * (255/input_range)).astype('uint8')
    cv2.imshow("source image", out)
    k = cv2.waitKey(0)
    cv2.destroyAllWindows()
    return k
    

def draw_output (img, label):

    Y_length, x_length = img.shape[0], img.shape[1]
    
    if label == 'QR':
        cv2.putText(img, 'QR', (20,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3, cv2.LINE_AA, 0)
    if label == 'empty':
        cv2.putText(img, 'empty', (20,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3, cv2.LINE_AA, 0)
    if label == 'horizontal':
        cv2.putText(img, 'horizontal', (20,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA, 0)
    if label == 'lef3':
        cv2.line(img, (0, 0), (0, Y_length), (0, 0, 255), 10)
    if label == 'left2':
        cv2.line(img, (round(x_length/6), 0), (round(x_length/6), Y_length), (0, 0, 255), 10)
    if label == 'left1':
        cv2.line(img, (round(x_length*2/6), 0), (round(x_length*2/6), Y_length), (0, 0, 255), 10)
    if label == 'center':
        cv2.line(img, (round(x_length*3/6), 0), (round(x_length*3/6), Y_length), (0, 0, 255), 10)
    if label == 'right1':
        cv2.line(img, (round(x_length*4/6), 0), (round(x_length*4/6), Y_length), (0, 0, 255), 10)
    if label == 'right2':
        cv2.line(img, (round(x_length*5/6), 0), (round(x_length*5/6), Y_length), (0, 0, 255), 10)
    if label == 'right3':
        cv2.line(img, (x_length, 0), (x_length, Y_length), (0, 0, 255), 10)
                    
    return img


    #(round(img.shape[1]/2), round(img.shape[0]/2))

def conv_str2list(string, labels):
    out= [0] * len(labels)
    out[labels.index(string)] = 1
    return out

def conv_list2str(Input : np.ndarray , labels):
    indexes = np.where(Input == np.amax(Input))
    return labels[indexes[0][0]]


### loading data

In [4]:
X_data = np.load('./../data/X_data.npy')
print(X_data)
print(X_data.shape)
# showing the processed image
show_processed_img(X_data[1])

Y_data = np.load('./../data/y_data.npy')
print(Y_data)
print(Y_data.shape)

Y_data = np.asarray([conv_str2list(i,labels=labels) for i in Y_data])
print(Y_data)
print(Y_data.shape)



[[11 12 12 ... 12 12 12]
 [ 5  5  5 ...  8  9  9]
 [10 10 10 ...  8  8  7]
 ...
 [14 13 14 ...  7  8  7]
 [ 5  6  5 ... 10  8  8]
 [12 13 13 ...  9  7  7]]
(4563, 400)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/raouf/.local/lib/python3.10/site-packages/cv2/qt/plugins"


['lef1' 'QR' 'empty' ... 'center' 'QR' 'QR']
(4563,)
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(4563, 10)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)


### building and training model

In [13]:
model = MLPClassifier(hidden_layer_sizes=(200,200,200,200),activation="relu" ,random_state=1, max_iter=2000)


train

In [14]:
model.fit(X_data,Y_data)

MLPClassifier(hidden_layer_sizes=(200, 200, 200, 200), max_iter=2000,
              random_state=1)

In [15]:
model.score(X_data, Y_data)

0.9066403681788298

### saving model

In [24]:
pickle.dump(model, open('model', 'wb'))

### loading learned model

In [4]:
model = pickle.load(open('model', 'rb'))

### using model and calculating the time of processing

In [8]:
img = cv2.imread("./../prepare_data/root_data/QR/QR_code.jpg",0) # you must load it as gray scal image
e1 = cv2.getTickCount()
features = preprocessing(img)
predictions = model.predict([features])
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()

out_label = conv_list2str(predictions[0], labels=labels)

print("prediction = " , out_label)
print("time of processing = ",time," s") # time of processing =  0.002  s

prediction =  QR
time of processing =  0.002788517  s


### using live from camera

In [9]:
camera = cv2.VideoCapture(0)
while (camera.isOpened):
    ret, frame = camera.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    features = preprocessing(img)
    predictions = model.predict([features])

    out_label = conv_list2str(predictions[0], labels=labels)

    # draw the output
    output = draw_output(frame, out_label)
    cv2.imshow("output", output)
    # print(out_label)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
cv2.destroyAllWindows()
camera.release()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/raouf/.local/lib/python3.10/site-packages/cv2/qt/plugins"
